# Feature Engineering for Load Forecasting
## German Energy Load Prediction Project

This notebook demonstrates the feature engineering process for predicting energy load 24 hours ahead using German energy data.

### 1. Initial Setup and Imports


In [4]:
# Standard library imports
import sys
import warnings
from pathlib import Path

# Third-party imports
## Data manipulation
import numpy as np
import pandas as pd
import holidays

## Machine learning
import lightgbm as lgb
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score
)
from sklearn.model_selection import TimeSeriesSplit

## Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning)

# Project path setup
current_dir = Path().absolute()
project_root = current_dir.parent if 'notebooks' in str(current_dir) else current_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
print(f"Project root added to path: {project_root}")

Project root added to path: d:\Projects\PORTF_2_German_Energy_forcast\german-energy-forcast


### 2. Feature Engineering Class Definition


In [2]:
class EnhancedFeatureExtractor24h:
    def __init__(self):
        self.windows = [24, 168, 336]  # 1 day, 1 week, 2 weeks
        self.morning_peak_hours = [7, 8, 9, 10, 11]
        self.evening_peak_hours = [16, 17, 18]
        self.base_hours = [0, 1, 2, 3, 4, 22, 23]
        
    def extract_features(self, df: pd.DataFrame) -> pd.DataFrame:
        try:
            df = df.copy()
            
            # Validate input
            if not isinstance(df.index, pd.DatetimeIndex):
                raise ValueError("DataFrame index must be DatetimeIndex")
            
            if 'load' not in df.columns:
                raise ValueError("DataFrame must contain 'load' column")
            
            # Basic time features with cyclical encoding
            df['hour'] = df.index.hour
            df['weekday'] = df.index.weekday
            df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
            df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
            df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
            df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
            
            # Add rolling statistics for each window
            for window in self.windows:
                df[f'load_{window}h_mean'] = df['load'].rolling(window=window, min_periods=1).mean()
                df[f'load_{window}h_std'] = df['load'].rolling(window=window, min_periods=1).std()
                df[f'load_{window}h_min'] = df['load'].rolling(window=window, min_periods=1).min()
                df[f'load_{window}h_max'] = df['load'].rolling(window=window, min_periods=1).max()
            
            # Add day-of-week specific features
            for day in range(7):
                mask = df.index.weekday == day
                df[f'load_day_{day}_mean'] = df[mask]['load'].rolling(window=24, min_periods=1).mean()
            
            # Add hour-of-day specific features
            for hour in range(24):
                mask = df.index.hour == hour
                df[f'load_hour_{hour}_mean'] = df[mask]['load'].rolling(window=7, min_periods=1).mean()
            
            # Add specialized peak handling
            df['is_morning_peak'] = df.index.hour.isin(self.morning_peak_hours)
            df['is_evening_peak'] = df.index.hour.isin(self.evening_peak_hours)
            df['is_base_load'] = df.index.hour.isin(self.base_hours)
            
            # Add more granular rolling stats for peak hours
            for window in [3, 6, 12]:
                # Morning peak specific
                morning_mask = df['is_morning_peak']
                df[f'morning_peak_{window}h_mean'] = df[morning_mask]['load'].rolling(window, min_periods=1).mean()
                df[f'morning_peak_{window}h_max'] = df[morning_mask]['load'].rolling(window, min_periods=1).max()
                
                # Evening peak specific
                evening_mask = df['is_evening_peak']
                df[f'evening_peak_{window}h_mean'] = df[evening_mask]['load'].rolling(window, min_periods=1).mean()
                df[f'evening_peak_{window}h_max'] = df[evening_mask]['load'].rolling(window, min_periods=1).max()
            
            # Add renewable features if available
            if 'solar_yesterday' in df.columns:
                df['solar_hour_sin'] = df['solar_yesterday'] * df['hour_sin']
                df['morning_solar_ramp'] = df['is_morning_peak'] * df['solar_yesterday']
                df['evening_solar_ramp'] = df['is_evening_peak'] * df['solar_yesterday']
                
                for window in [24, 168]:
                    df[f'solar_{window}h_mean'] = df['solar_yesterday'].rolling(window, min_periods=1).mean()
            
            if all(col in df.columns for col in ['wind_offshore_yesterday', 'wind_onshore_yesterday']):
                df['total_wind'] = df['wind_offshore_yesterday'] + df['wind_onshore_yesterday']
                df['wind_hour_sin'] = df['total_wind'] * df['hour_sin']
                
                for window in [24, 168]:
                    df[f'wind_{window}h_mean'] = df['total_wind'].rolling(window, min_periods=1).mean()
            
            # Clean up temporary columns
            columns_to_drop = ['hour', 'weekday']
            df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
            
            return df
            
        except Exception as e:
            print(f"Error in feature extraction: {str(e)}")
            raise

def create_specialized_model(model_type: str):
    """Create a model with parameters optimized for specific time periods"""
    if model_type == 'morning_peak':
        return lgb.LGBMRegressor(
            n_estimators=1200,
            learning_rate=0.003,
            max_depth=9,
            num_leaves=60,
            subsample=0.7,
            colsample_bytree=0.7,
            reg_alpha=0.5,
            reg_lambda=0.5,
            random_state=42,
            n_jobs=-1
        )
    elif model_type == 'evening_peak':
        return lgb.LGBMRegressor(
            n_estimators=1200,
            learning_rate=0.003,
            max_depth=9,
            num_leaves=60,
            subsample=0.7,
            colsample_bytree=0.7,
            reg_alpha=0.5,
            reg_lambda=0.5,
            random_state=42,
            n_jobs=-1
        )
    else:  # base_load
        return lgb.LGBMRegressor(
            n_estimators=800,
            learning_rate=0.005,
            max_depth=7,
            num_leaves=40,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.3,
            reg_lambda=0.3,
            random_state=42,
            n_jobs=-1
        )

def train_specialized_models(X_train, y_train):
    """Train separate models for different parts of the day"""
    try:
        # Split data by time periods
        morning_peak_mask = X_train.index.hour.isin(range(7, 12))
        evening_peak_mask = X_train.index.hour.isin(range(16, 19))
        base_load_mask = ~(morning_peak_mask | evening_peak_mask)
        
        # Create and train specialized models
        models = {}
        for model_type, mask in [
            ('morning_peak', morning_peak_mask),
            ('evening_peak', evening_peak_mask),
            ('base_load', base_load_mask)
        ]:
            model = create_specialized_model(model_type)
            model.fit(X_train[mask], y_train[mask])
            models[model_type] = model
        
        return models
        
    except Exception as e:
        print(f"Error in training specialized models: {str(e)}")
        raise

def predict_with_specialized_models(models, X_test):
    """Make predictions using specialized models based on hour of day"""
    try:
        predictions = pd.Series(index=X_test.index, dtype=float)
        
        for hour in X_test.index.hour.unique():
            if hour in range(7, 12):
                model = models['morning_peak']
            elif hour in range(16, 19):
                model = models['evening_peak']
            else:
                model = models['base_load']
                
            mask = X_test.index.hour == hour
            predictions[mask] = model.predict(X_test[mask])
            
        return predictions
        
    except Exception as e:
        print(f"Error in making predictions: {str(e)}")
        raise

def analyze_results(y_test, predictions):
    """Analyze model performance in detail"""
    try:
        # Calculate overall metrics
        mae = mean_absolute_error(y_test, predictions)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        r2 = r2_score(y_test, predictions)
        mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100
        
        # Hourly analysis
        hourly_performance = pd.DataFrame({
            'hour': y_test.index.hour,
            'actual': y_test,
            'predicted': predictions,
            'error': y_test - predictions,
            'abs_error': abs(y_test - predictions)
        })
        
        hourly_stats = hourly_performance.groupby('hour').agg({
            'error': ['mean', 'std'],
            'abs_error': 'mean'
        })
        
        # Peak vs Off-peak analysis
        peak_hours = list(range(7, 12)) + list(range(16, 19))
        hourly_performance['is_peak'] = hourly_performance['hour'].isin(peak_hours)
        peak_stats = hourly_performance.groupby('is_peak')['abs_error'].mean()
        
        return {
            'metrics': {
                'mae': mae,
                'rmse': rmse,
                'r2': r2,
                'mape': mape
            },
            'hourly_stats': hourly_stats,
            'peak_stats': peak_stats
        }
        
    except Exception as e:
        print(f"Error in analyzing results: {str(e)}")
        raise

def plot_results(y_test, predictions, analysis):
    """Create visualization of results"""
    try:
        # Create figure with secondary y-axis
        fig = make_subplots(rows=2, cols=1,
                           subplot_titles=('Actual vs Predicted Load', 'Prediction Error'),
                           vertical_spacing=0.16,
                           row_heights=[0.7, 0.3])
        
        # Add traces for actual and predicted values
        fig.add_trace(
            go.Scatter(x=y_test.index, y=y_test, name="Actual Load",
                      line=dict(color='#636EFA', width=2)),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(x=y_test.index, y=predictions, name="Predicted Load",
                      line=dict(color='#EF553B', width=2, dash='dash')),
            row=1, col=1
        )
        
        # Add error plot
        error = y_test - predictions
        fig.add_trace(
            go.Scatter(x=y_test.index, y=error, name="Prediction Error",
                      line=dict(color='#00CC96', width=1.5)),
            row=2, col=1
        )
        
        # Update layout
        fig.update_layout(
            title_text="24-Hour Load Forecast",
            template='plotly_dark',
            showlegend=True,
            height=800
        )
        
        return fig
        
    except Exception as e:
        print(f"Error in plotting results: {str(e)}")
        raise

def main(df, train_end_date, test_end_date):
    """Main function to run the entire pipeline"""
    try:
        # Prepare data
        train_data = df[df.index <= train_end_date].copy()
        test_data = df[(df.index > train_end_date) & (df.index <= test_end_date)].copy()
        
        # Extract features
        feature_extractor = EnhancedFeatureExtractor24h()
        X_train = feature_extractor.extract_features(train_data)
        X_test = feature_extractor.extract_features(test_data)
        
        y_train = train_data['load']
        y_test = test_data['load']
        
        # Train models
        models = train_specialized_models(X_train, y_train)
        
        # Make predictions
        predictions = predict_with_specialized_models(models, X_test)
        
        # Analyze results
        analysis = analyze_results(y_test, predictions)
        
        # Plot results
        fig = plot_results(y_test, predictions, analysis)
        
        # Print metrics
        print("\nModel Performance:")
        print(f"MAE: {analysis['metrics']['mae']:.3f}")
        print(f"RMSE: {analysis['metrics']['rmse']:.3f}")
        print(f"R2 Score: {analysis['metrics']['r2']:.3f}")
        print(f"MAPE: {analysis['metrics']['mape']:.2f}%")
        
        print("\nPeak vs Off-peak Performance:")
        print(analysis['peak_stats'])
        
        return {
            'models': models,
            'predictions': predictions,
            'analysis': analysis,
            'plot': fig,
            'y_test': y_test,  # Added this
            'test_data': test_data  # Added this for solar analysis
        }
        
    except Exception as e:
        print(f"Error in main pipeline: {str(e)}")
        raise


def analyze_worst_periods(y_test, predictions, n_worst=5):
    """Analyze the periods with highest prediction errors"""
    
    # Calculate absolute errors
    errors = pd.DataFrame({
        'actual': y_test,
        'predicted': predictions,
        'abs_error': abs(y_test - predictions),
        'error': y_test - predictions,
        'hour': y_test.index.hour,
        'weekday': y_test.index.weekday,
        'date': y_test.index.date
    })
    
    # Find worst days by average daily error
    daily_errors = errors.groupby('date')['abs_error'].mean().sort_values(ascending=False)
    worst_days = daily_errors.head(n_worst)
    
    print("\nWorst Performing Days:")
    for date, error in worst_days.items():
        day_data = errors[errors['date'] == date]
        max_error_hour = day_data.loc[day_data['abs_error'].idxmax(), 'hour']
        print(f"\nDate: {date}, Avg Error: {error:.2f} MW")
        print(f"Max Error Hour: {max_error_hour}:00")
        print(f"Weekday: {['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'][day_data['weekday'].iloc[0]]}")
    
    # Create visualization of worst days
    fig = go.Figure()
    
    for date in worst_days.index:
        mask = errors['date'] == date
        day_data = errors[mask]
        
        # Actual values
        fig.add_trace(go.Scatter(
            x=day_data.index,
            y=day_data['actual'],
            name=f'Actual {date}',
            line=dict(width=2)
        ))
        
        # Predicted values
        fig.add_trace(go.Scatter(
            x=day_data.index,
            y=day_data['predicted'],
            name=f'Predicted {date}',
            line=dict(dash='dash', width=2)
        ))
    
    fig.update_layout(
        title='Worst Performing Days Analysis',
        xaxis_title='Time',
        yaxis_title='Load (MW)',
        template='plotly_dark',
        height=600,
        showlegend=True
    )
    
    # Additional error patterns analysis
    print("\nError Patterns Analysis:")
    
    # Time-based patterns
    hour_errors = errors.groupby('hour')['abs_error'].mean()
    worst_hours = hour_errors.nlargest(3)
    print("\nWorst Performing Hours:")
    for hour, error in worst_hours.items():
        print(f"Hour {hour}:00 - Avg Error: {error:.2f} MW")
    
    # Day of week patterns
    weekday_errors = errors.groupby('weekday')['abs_error'].mean()
    worst_weekdays = weekday_errors.nlargest(3)
    print("\nWorst Performing Weekdays:")
    for day, error in worst_weekdays.items():
        print(f"{['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'][day]} - Avg Error: {error:.2f} MW")
    
    # Calculate error statistics for different load levels
    errors['load_level'] = pd.qcut(errors['actual'], q=3, labels=['Low', 'Medium', 'High'])
    load_level_errors = errors.groupby('load_level')['abs_error'].mean()
    print("\nErrors by Load Level:")
    print(load_level_errors)
    
    return {
        'worst_days': worst_days,
        'hour_errors': hour_errors,
        'weekday_errors': weekday_errors,
        'load_level_errors': load_level_errors,
        'plot': fig
    }


# If solar data is available, analyze its relationship with errors
def analyze_solar_impact(df, errors):
    if 'solar_yesterday' in df.columns:
        solar_data = df['solar_yesterday']
        solar_errors = pd.DataFrame({
            'solar': solar_data,
            'error': errors,
            'hour': df.index.hour
        })
        
        # Analyze errors during solar ramp-up and ramp-down
        morning_ramp = solar_errors[solar_errors['hour'].isin(range(6, 11))]
        evening_ramp = solar_errors[solar_errors['hour'].isin(range(16, 21))]
        
        print("\nSolar Impact Analysis:")
        print(f"Morning Ramp Average Error: {morning_ramp['error'].abs().mean():.2f} MW")
        print(f"Evening Ramp Average Error: {evening_ramp['error'].abs().mean():.2f} MW")
        
        # Correlation analysis
        correlation = solar_errors['solar'].corr(solar_errors['error'].abs())
        print(f"Solar-Error Correlation: {correlation:.3f}")
        
        return {
            'morning_ramp_error': morning_ramp['error'].abs().mean(),
            'evening_ramp_error': evening_ramp['error'].abs().mean(),
            'correlation': correlation
        }
    
def analyze_error_distribution(y_test, predictions):
    errors = abs(y_test - predictions)
    percentage_errors = (errors / y_test) * 100

    print("\nError Distribution Analysis:")
    print(f"Mean Error: {errors.mean():.2f} MW ({percentage_errors.mean():.2f}%)")
    print(f"Median Error: {errors.median():.2f} MW ({percentage_errors.median():.2f}%)")
    print(f"90th Percentile Error: {errors.quantile(0.9):.2f} MW ({percentage_errors.quantile(0.9):.2f}%)")

    # Create histogram of percentage errors
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=percentage_errors, nbinsx=50))
    fig.update_layout(
        title='Distribution of Percentage Errors',
        xaxis_title='Percentage Error (%)',
        yaxis_title='Count',
        template='plotly_dark'
    )
    return fig




In [5]:
# Load data
from src.data.data_loader import LoadDataLoader
loader = LoadDataLoader()
processed_data = loader.load_processed_data()

# Load renewables data and handle timezones properly
renewables_df = pd.read_csv('../data/raw/renewables_20230101_20240301.csv', 
                           parse_dates=['Unnamed: 0'],
                           index_col='Unnamed: 0')

# First convert to UTC, then remove timezone
renewables_df.index = pd.to_datetime(renewables_df.index, utc=True).tz_localize(None)

# Print column names to debug
print("Renewables columns:", renewables_df.columns.tolist())

def prepare_feature_df(load_data, renewables_data):
    """
    Prepare feature DataFrame from load and renewables data
    """
    try:
        # Create copies to avoid modifying original data
        load_data = load_data.copy()
        renewables_data = renewables_data.copy()
        
        # Ensure datetime indices
        if not isinstance(load_data.index, pd.DatetimeIndex):
            load_data.index = pd.to_datetime(load_data.index, utc=True)
        if not isinstance(renewables_data.index, pd.DatetimeIndex):
            renewables_data.index = pd.to_datetime(renewables_data.index, utc=True)
        
        # Convert to timezone-naive
        if load_data.index.tz is not None:
            load_data.index = load_data.index.tz_localize(None)
        if renewables_data.index.tz is not None:
            renewables_data.index = renewables_data.index.tz_localize(None)
        
        # Resample load data to hourly if needed
        load_hourly = load_data.resample('h').mean()
        
        # Create initial feature DataFrame
        df = pd.DataFrame(index=load_hourly.index)
        df['load'] = load_hourly.values
        
        # Add time-based features
        df['hour'] = df.index.hour
        df['weekday'] = df.index.weekday
        df['month'] = df.index.month
        df['is_weekend'] = df.index.weekday >= 5
        df['is_holiday'] = df.index.isin(holidays.US().keys())
        
        # Add cyclical encoding
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
        df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
        df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
        
        # Add lagged features
        df['load_same_hour_yesterday'] = df['load'].shift(24)
        df['load_same_hour_lastweek'] = df['load'].shift(168)
        
        # Add trend features
        df['load_24h_trend'] = df['load'] - df['load'].shift(24)
        df['load_week_trend'] = df['load'] - df['load'].shift(168)
        df['load_volatility_24h'] = df['load'].rolling(window=24).std()
        
        # Add renewables features if available
        if renewables_data is not None:
            try:
                # Print available columns
                print("Available renewables columns:", renewables_data.columns.tolist())
                
                # Check column names and map them if needed
                solar_col = next((col for col in renewables_data.columns if 'solar' in col.lower()), None)
                wind_offshore_col = next((col for col in renewables_data.columns if 'offshore' in col.lower()), None)
                wind_onshore_col = next((col for col in renewables_data.columns if 'onshore' in col.lower()), None)
                
                if solar_col:
                    df['solar_yesterday'] = renewables_data[solar_col].shift(24)
                if wind_offshore_col:
                    df['wind_offshore_yesterday'] = renewables_data[wind_offshore_col].shift(24)
                if wind_onshore_col:
                    df['wind_onshore_yesterday'] = renewables_data[wind_onshore_col].shift(24)
                    
            except Exception as e:
                print(f"Warning: Could not process renewables data: {str(e)}")
        
        # Add seasonal features
        df['season_spring'] = df.index.month.isin([3, 4, 5])
        df['season_summer'] = df.index.month.isin([6, 7, 8])
        df['season_fall'] = df.index.month.isin([9, 10, 11])
        df['season_winter'] = df.index.month.isin([12, 1, 2])
        
        # Add part of day features
        df['part_of_day_morning'] = df.index.hour.isin([6, 7, 8, 9, 10, 11])
        df['part_of_day_afternoon'] = df.index.hour.isin([12, 13, 14, 15, 16, 17])
        df['part_of_day_evening'] = df.index.hour.isin([18, 19, 20, 21, 22, 23])
        df['part_of_day_night'] = df.index.hour.isin([0, 1, 2, 3, 4, 5])

        # Rolling statistics
        df['load_7d_mean'] = df['load'].rolling(window=168).mean()
        df['load_7d_std'] = df['load'].rolling(window=168).std()
        
        # Time interaction features
        df['load_hour_interaction'] = df['load'] * df['hour_sin']
        df['load_weekday_interaction'] = df['load'] * df['weekday_sin']
        
        return df
        
    except Exception as e:
        print(f"Error in prepare_feature_df: {str(e)}")
        raise

# Prepare feature DataFrame
df = prepare_feature_df(processed_data, renewables_df)

# Remove any NaN values and print shape
df = df.dropna()
print(f"\nDataFrame shape after preparing features: {df.shape}")

Initialized LoadDataLoader with raw data path: ..\data\raw

Loaded processed data:
Shape: (40799, 1)
Columns: Actual Load
Date range: 2022-12-31 23:00:00 to 2024-02-29 22:45:00
Renewables columns: ['Solar', 'Wind Offshore', 'Wind Onshore']
Available renewables columns: ['Solar', 'Wind Offshore', 'Wind Onshore']

DataFrame shape after preparing features: (10030, 30)


In [7]:
# Define train/test split dates
train_end_date = '2023-12-24'
test_end_date = '2024-01-25'

# Run the complete analysis pipeline
results = main(df, train_end_date, test_end_date)
results['plot'].show()

# Now run the worst periods analysis
worst_periods_analysis = analyze_worst_periods(results['y_test'], results['predictions'])
worst_periods_analysis['plot'].show()

# Run solar analysis if available
if 'solar_yesterday' in df.columns:
    solar_analysis = analyze_solar_impact(results['test_data'], 
                                        results['y_test'] - results['predictions'])
    
# Add this analysis to your code
error_dist_plot = analyze_error_distribution(results['y_test'], results['predictions'])
error_dist_plot.show()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000955 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10136
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 61
[LightGBM] [Info] Start training from score 59587.594000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g


Worst Performing Days:

Date: 2024-01-15, Avg Error: 1269.29 MW
Max Error Hour: 10:00
Weekday: Mon

Date: 2023-12-25, Avg Error: 1014.40 MW
Max Error Hour: 6:00
Weekday: Mon

Date: 2024-01-17, Avg Error: 993.45 MW
Max Error Hour: 8:00
Weekday: Wed

Date: 2024-01-08, Avg Error: 945.91 MW
Max Error Hour: 6:00
Weekday: Mon

Date: 2024-01-19, Avg Error: 871.89 MW
Max Error Hour: 9:00
Weekday: Fri

Error Patterns Analysis:

Worst Performing Hours:
Hour 7:00 - Avg Error: 974.39 MW
Hour 12:00 - Avg Error: 879.26 MW
Hour 16:00 - Avg Error: 845.36 MW

Worst Performing Weekdays:
Monday - Avg Error: 884.29 MW
Friday - Avg Error: 483.10 MW
Wednesday - Avg Error: 461.33 MW

Errors by Load Level:
load_level
Low       374.903466
Medium    282.755450
High      790.032996
Name: abs_error, dtype: float64


C:\Users\risha\AppData\Local\Temp\ipykernel_10948\67300156.py:391: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




Solar Impact Analysis:
Morning Ramp Average Error: 768.57 MW
Evening Ramp Average Error: 531.82 MW
Solar-Error Correlation: 0.062

Error Distribution Analysis:
Mean Error: 482.82 MW (0.82%)
Median Error: 257.15 MW (0.48%)
90th Percentile Error: 1131.47 MW (1.89%)


In [6]:
class EnhancedFeatureExtractor24h:
    def __init__(self):
        self.windows = [24, 168, 336]  # 1 day, 1 week, 2 weeks
        self.morning_peak_hours = [7, 8, 9, 10, 11]
        self.evening_peak_hours = [16, 17, 18]
        self.base_hours = [0, 1, 2, 3, 4, 22, 23]
        
    def extract_features(self, df: pd.DataFrame) -> pd.DataFrame:
        try:
            df = df.copy()
            
            # Validate input
            if not isinstance(df.index, pd.DatetimeIndex):
                raise ValueError("DataFrame index must be DatetimeIndex")
            
            if 'load' not in df.columns:
                raise ValueError("DataFrame must contain 'load' column")
            
            # Basic time features with cyclical encoding
            df['hour'] = df.index.hour
            df['weekday'] = df.index.weekday
            df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
            df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
            df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
            df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
            
            # Add rolling statistics for each window
            for window in self.windows:
                df[f'load_{window}h_mean'] = df['load'].rolling(window=window, min_periods=1).mean()
                df[f'load_{window}h_std'] = df['load'].rolling(window=window, min_periods=1).std()
                df[f'load_{window}h_min'] = df['load'].rolling(window=window, min_periods=1).min()
                df[f'load_{window}h_max'] = df['load'].rolling(window=window, min_periods=1).max()
            
            # Add day-of-week specific features
            for day in range(7):
                mask = df.index.weekday == day
                df[f'load_day_{day}_mean'] = df[mask]['load'].rolling(window=24, min_periods=1).mean()
            
            # Add hour-of-day specific features
            for hour in range(24):
                mask = df.index.hour == hour
                df[f'load_hour_{hour}_mean'] = df[mask]['load'].rolling(window=7, min_periods=1).mean()
            
            # Add specialized peak handling
            df['is_morning_peak'] = df.index.hour.isin(self.morning_peak_hours)
            df['is_evening_peak'] = df.index.hour.isin(self.evening_peak_hours)
            df['is_base_load'] = df.index.hour.isin(self.base_hours)
            
            # Add more granular rolling stats for peak hours
            for window in [3, 6, 12]:
                # Morning peak specific
                morning_mask = df['is_morning_peak']
                df[f'morning_peak_{window}h_mean'] = df[morning_mask]['load'].rolling(window, min_periods=1).mean()
                df[f'morning_peak_{window}h_max'] = df[morning_mask]['load'].rolling(window, min_periods=1).max()
                
                # Evening peak specific
                evening_mask = df['is_evening_peak']
                df[f'evening_peak_{window}h_mean'] = df[evening_mask]['load'].rolling(window, min_periods=1).mean()
                df[f'evening_peak_{window}h_max'] = df[evening_mask]['load'].rolling(window, min_periods=1).max()
            
            # Add renewable features if available
            if 'solar_yesterday' in df.columns:
                df['solar_hour_sin'] = df['solar_yesterday'] * df['hour_sin']
                df['morning_solar_ramp'] = df['is_morning_peak'] * df['solar_yesterday']
                df['evening_solar_ramp'] = df['is_evening_peak'] * df['solar_yesterday']
                
                for window in [24, 168]:
                    df[f'solar_{window}h_mean'] = df['solar_yesterday'].rolling(window, min_periods=1).mean()
            
            if all(col in df.columns for col in ['wind_offshore_yesterday', 'wind_onshore_yesterday']):
                df['total_wind'] = df['wind_offshore_yesterday'] + df['wind_onshore_yesterday']
                df['wind_hour_sin'] = df['total_wind'] * df['hour_sin']
                
                for window in [24, 168]:
                    df[f'wind_{window}h_mean'] = df['total_wind'].rolling(window, min_periods=1).mean()
            
            # Clean up temporary columns
            columns_to_drop = ['hour', 'weekday']
            df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
            
            return df
            
        except Exception as e:
            print(f"Error in feature extraction: {str(e)}")
            raise

def create_specialized_model(model_type: str):
    """Create a model with parameters optimized for specific time periods"""
    if model_type == 'morning_peak':
        return lgb.LGBMRegressor(
            n_estimators=1200,
            learning_rate=0.003,
            max_depth=9,
            num_leaves=60,
            subsample=0.7,
            colsample_bytree=0.7,
            reg_alpha=0.5,
            reg_lambda=0.5,
            random_state=42,
            n_jobs=-1
        )
    elif model_type == 'evening_peak':
        return lgb.LGBMRegressor(
            n_estimators=1200,
            learning_rate=0.003,
            max_depth=9,
            num_leaves=60,
            subsample=0.7,
            colsample_bytree=0.7,
            reg_alpha=0.5,
            reg_lambda=0.5,
            random_state=42,
            n_jobs=-1
        )
    else:  # base_load
        return lgb.LGBMRegressor(
            n_estimators=800,
            learning_rate=0.005,
            max_depth=7,
            num_leaves=40,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.3,
            reg_lambda=0.3,
            random_state=42,
            n_jobs=-1
        )

def train_specialized_models(X_train, y_train):
    """Train separate models for different parts of the day"""
    try:
        # Split data by time periods
        morning_peak_mask = X_train.index.hour.isin(range(7, 12))
        evening_peak_mask = X_train.index.hour.isin(range(16, 19))
        base_load_mask = ~(morning_peak_mask | evening_peak_mask)
        
        # Create and train specialized models
        models = {}
        for model_type, mask in [
            ('morning_peak', morning_peak_mask),
            ('evening_peak', evening_peak_mask),
            ('base_load', base_load_mask)
        ]:
            model = create_specialized_model(model_type)
            model.fit(X_train[mask], y_train[mask])
            models[model_type] = model
        
        return models
        
    except Exception as e:
        print(f"Error in training specialized models: {str(e)}")
        raise

def predict_with_specialized_models(models, X_test):
    """Make predictions using specialized models based on hour of day"""
    try:
        predictions = pd.Series(index=X_test.index, dtype=float)
        
        for hour in X_test.index.hour.unique():
            if hour in range(7, 12):
                model = models['morning_peak']
            elif hour in range(16, 19):
                model = models['evening_peak']
            else:
                model = models['base_load']
                
            mask = X_test.index.hour == hour
            predictions[mask] = model.predict(X_test[mask])
            
        return predictions
        
    except Exception as e:
        print(f"Error in making predictions: {str(e)}")
        raise

def analyze_results(y_test, predictions):
    """Analyze model performance in detail"""
    try:
        # Calculate overall metrics
        mae = mean_absolute_error(y_test, predictions)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        r2 = r2_score(y_test, predictions)
        mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100
        
        # Hourly analysis
        hourly_performance = pd.DataFrame({
            'hour': y_test.index.hour,
            'actual': y_test,
            'predicted': predictions,
            'error': y_test - predictions,
            'abs_error': abs(y_test - predictions)
        })
        
        hourly_stats = hourly_performance.groupby('hour').agg({
            'error': ['mean', 'std'],
            'abs_error': 'mean'
        })
        
        # Peak vs Off-peak analysis
        peak_hours = list(range(7, 12)) + list(range(16, 19))
        hourly_performance['is_peak'] = hourly_performance['hour'].isin(peak_hours)
        peak_stats = hourly_performance.groupby('is_peak')['abs_error'].mean()
        
        return {
            'metrics': {
                'mae': mae,
                'rmse': rmse,
                'r2': r2,
                'mape': mape
            },
            'hourly_stats': hourly_stats,
            'peak_stats': peak_stats
        }
        
    except Exception as e:
        print(f"Error in analyzing results: {str(e)}")
        raise

def plot_results(y_test, predictions, analysis):
    """Create visualization of results"""
    try:
        # Create figure with secondary y-axis
        fig = make_subplots(rows=2, cols=1,
                           subplot_titles=('Actual vs Predicted Load', 'Prediction Error'),
                           vertical_spacing=0.16,
                           row_heights=[0.7, 0.3])
        
        # Add traces for actual and predicted values
        fig.add_trace(
            go.Scatter(x=y_test.index, y=y_test, name="Actual Load",
                      line=dict(color='#636EFA', width=2)),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(x=y_test.index, y=predictions, name="Predicted Load",
                      line=dict(color='#EF553B', width=2, dash='dash')),
            row=1, col=1
        )
        
        # Add error plot
        error = y_test - predictions
        fig.add_trace(
            go.Scatter(x=y_test.index, y=error, name="Prediction Error",
                      line=dict(color='#00CC96', width=1.5)),
            row=2, col=1
        )
        
        # Update layout
        fig.update_layout(
            title_text="24-Hour Load Forecast",
            template='plotly_dark',
            showlegend=True,
            height=800
        )
        
        return fig
        
    except Exception as e:
        print(f"Error in plotting results: {str(e)}")
        raise

def main(df, train_end_date, test_end_date):
    """Main function to run the entire pipeline"""
    try:
        # Prepare data
        train_data = df[df.index <= train_end_date].copy()
        test_data = df[(df.index > train_end_date) & (df.index <= test_end_date)].copy()
        
        # Extract features
        feature_extractor = EnhancedFeatureExtractor24h()
        X_train = feature_extractor.extract_features(train_data)
        X_test = feature_extractor.extract_features(test_data)
        
        y_train = train_data['load']
        y_test = test_data['load']
        
        # Train models
        models = train_specialized_models(X_train, y_train)
        
        # Make predictions
        predictions = predict_with_specialized_models(models, X_test)
        
        # Analyze results
        analysis = analyze_results(y_test, predictions)
        
        # Plot results
        fig = plot_results(y_test, predictions, analysis)
        
        # Print metrics
        print("\nModel Performance:")
        print(f"MAE: {analysis['metrics']['mae']:.3f}")
        print(f"RMSE: {analysis['metrics']['rmse']:.3f}")
        print(f"R2 Score: {analysis['metrics']['r2']:.3f}")
        print(f"MAPE: {analysis['metrics']['mape']:.2f}%")
        
        print("\nPeak vs Off-peak Performance:")
        print(analysis['peak_stats'])
        
        return {
            'models': models,
            'predictions': predictions,
            'analysis': analysis,
            'plot': fig,
            'y_test': y_test,  # Added this
            'test_data': test_data  # Added this for solar analysis
        }
        
    except Exception as e:
        print(f"Error in main pipeline: {str(e)}")
        raise


def analyze_worst_periods(y_test, predictions, n_worst=5):
    """Analyze the periods with highest prediction errors"""
    
    # Calculate absolute errors
    errors = pd.DataFrame({
        'actual': y_test,
        'predicted': predictions,
        'abs_error': abs(y_test - predictions),
        'error': y_test - predictions,
        'hour': y_test.index.hour,
        'weekday': y_test.index.weekday,
        'date': y_test.index.date
    })
    
    # Find worst days by average daily error
    daily_errors = errors.groupby('date')['abs_error'].mean().sort_values(ascending=False)
    worst_days = daily_errors.head(n_worst)
    
    print("\nWorst Performing Days:")
    for date, error in worst_days.items():
        day_data = errors[errors['date'] == date]
        max_error_hour = day_data.loc[day_data['abs_error'].idxmax(), 'hour']
        print(f"\nDate: {date}, Avg Error: {error:.2f} MW")
        print(f"Max Error Hour: {max_error_hour}:00")
        print(f"Weekday: {['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'][day_data['weekday'].iloc[0]]}")
    
    # Create visualization of worst days
    fig = go.Figure()
    
    for date in worst_days.index:
        mask = errors['date'] == date
        day_data = errors[mask]
        
        # Actual values
        fig.add_trace(go.Scatter(
            x=day_data.index,
            y=day_data['actual'],
            name=f'Actual {date}',
            line=dict(width=2)
        ))
        
        # Predicted values
        fig.add_trace(go.Scatter(
            x=day_data.index,
            y=day_data['predicted'],
            name=f'Predicted {date}',
            line=dict(dash='dash', width=2)
        ))
    
    fig.update_layout(
        title='Worst Performing Days Analysis',
        xaxis_title='Time',
        yaxis_title='Load (MW)',
        template='plotly_dark',
        height=600,
        showlegend=True
    )
    
    # Additional error patterns analysis
    print("\nError Patterns Analysis:")
    
    # Time-based patterns
    hour_errors = errors.groupby('hour')['abs_error'].mean()
    worst_hours = hour_errors.nlargest(3)
    print("\nWorst Performing Hours:")
    for hour, error in worst_hours.items():
        print(f"Hour {hour}:00 - Avg Error: {error:.2f} MW")
    
    # Day of week patterns
    weekday_errors = errors.groupby('weekday')['abs_error'].mean()
    worst_weekdays = weekday_errors.nlargest(3)
    print("\nWorst Performing Weekdays:")
    for day, error in worst_weekdays.items():
        print(f"{['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'][day]} - Avg Error: {error:.2f} MW")
    
    # Calculate error statistics for different load levels
    errors['load_level'] = pd.qcut(errors['actual'], q=3, labels=['Low', 'Medium', 'High'])
    load_level_errors = errors.groupby('load_level')['abs_error'].mean()
    print("\nErrors by Load Level:")
    print(load_level_errors)
    
    return {
        'worst_days': worst_days,
        'hour_errors': hour_errors,
        'weekday_errors': weekday_errors,
        'load_level_errors': load_level_errors,
        'plot': fig
    }


# If solar data is available, analyze its relationship with errors
def analyze_solar_impact(df, errors):
    if 'solar_yesterday' in df.columns:
        solar_data = df['solar_yesterday']
        solar_errors = pd.DataFrame({
            'solar': solar_data,
            'error': errors,
            'hour': df.index.hour
        })
        
        # Analyze errors during solar ramp-up and ramp-down
        morning_ramp = solar_errors[solar_errors['hour'].isin(range(6, 11))]
        evening_ramp = solar_errors[solar_errors['hour'].isin(range(16, 21))]
        
        print("\nSolar Impact Analysis:")
        print(f"Morning Ramp Average Error: {morning_ramp['error'].abs().mean():.2f} MW")
        print(f"Evening Ramp Average Error: {evening_ramp['error'].abs().mean():.2f} MW")
        
        # Correlation analysis
        correlation = solar_errors['solar'].corr(solar_errors['error'].abs())
        print(f"Solar-Error Correlation: {correlation:.3f}")
        
        return {
            'morning_ramp_error': morning_ramp['error'].abs().mean(),
            'evening_ramp_error': evening_ramp['error'].abs().mean(),
            'correlation': correlation
        }
    
def analyze_error_distribution(y_test, predictions):
    errors = abs(y_test - predictions)
    percentage_errors = (errors / y_test) * 100

    print("\nError Distribution Analysis:")
    print(f"Mean Error: {errors.mean():.2f} MW ({percentage_errors.mean():.2f}%)")
    print(f"Median Error: {errors.median():.2f} MW ({percentage_errors.median():.2f}%)")
    print(f"90th Percentile Error: {errors.quantile(0.9):.2f} MW ({percentage_errors.quantile(0.9):.2f}%)")

    # Create histogram of percentage errors
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=percentage_errors, nbinsx=50))
    fig.update_layout(
        title='Distribution of Percentage Errors',
        xaxis_title='Percentage Error (%)',
        yaxis_title='Count',
        template='plotly_dark'
    )
    return fig




[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000963 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10171
[LightGBM] [Info] Number of data points in the train set: 1785, number of used features: 61
[LightGBM] [Info] Start training from score 59423.685994
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g


Worst Performing Days:

Date: 2024-01-15, Avg Error: 1247.08 MW
Max Error Hour: 10:00
Weekday: Mon

Date: 2024-01-17, Avg Error: 1016.84 MW
Max Error Hour: 8:00
Weekday: Wed

Date: 2024-01-08, Avg Error: 955.01 MW
Max Error Hour: 6:00
Weekday: Mon

Date: 2024-01-19, Avg Error: 878.89 MW
Max Error Hour: 9:00
Weekday: Fri

Date: 2024-01-13, Avg Error: 616.06 MW
Max Error Hour: 7:00
Weekday: Sat

Error Patterns Analysis:

Worst Performing Hours:
Hour 7:00 - Avg Error: 1038.82 MW
Hour 16:00 - Avg Error: 841.04 MW
Hour 17:00 - Avg Error: 772.61 MW

Worst Performing Weekdays:
Monday - Avg Error: 749.57 MW
Friday - Avg Error: 491.84 MW
Wednesday - Avg Error: 468.19 MW

Errors by Load Level:
load_level
Low       216.589813
Medium    388.798095
High      735.355320
Name: abs_error, dtype: float64


C:\Users\risha\AppData\Local\Temp\ipykernel_10948\67300156.py:391: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




Solar Impact Analysis:
Morning Ramp Average Error: 742.81 MW
Evening Ramp Average Error: 513.39 MW
Solar-Error Correlation: 0.050

Error Distribution Analysis:
Mean Error: 446.99 MW (0.71%)
Median Error: 251.94 MW (0.45%)
90th Percentile Error: 1076.73 MW (1.63%)


### 2. Feature Engineering Pipeline

Our feature set consists of three main categories:
1. **Temporal Features**: Calendar-based patterns (weekends, holidays, seasons)
2. **Load History**: Previous day and week load patterns
3. **Renewable Energy**: Solar and wind generation patterns

#### 2.1 Temporal Feature Creation

In [13]:
import plotly.graph_objects as go

def plot_feature_importance(X_train, y_train):
    from sklearn.ensemble import RandomForestRegressor
    
    # Train a simple RF model to get feature importance
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    
    # Create importance DataFrame
    importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': rf.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    # Create Plotly figure
    fig = go.Figure()
    
    # Add bar plot
    fig.add_trace(
        go.Bar(
            x=importance_df['Importance'],
            y=importance_df['Feature'],
            orientation='h',
            marker=dict(
                color='#636EFA',
                line=dict(color='#EF553B', width=1)
            )
        )
    )
    
    # Update layout
    fig.update_layout(
        template='plotly_dark',
        title='Feature Importance for 24-Hour Load Prediction',
        xaxis_title='Importance',
        yaxis_title='Feature',
        height=600,
        yaxis=dict(autorange="reversed"),  # To match the original order
        margin=dict(l=200)  # Add more margin for feature names
    )
    
    fig.show()
    
    # Print top 5 features
    print("\nTop 5 Most Important Features:")
    print(importance_df.head().to_string(index=False))
    
    return importance_df

# Generate and display feature importance
importance_df = plot_feature_importance(X_train, y_train)


Top 5 Most Important Features:
                 Feature  Importance
 load_same_hour_lastweek    0.501125
         load_week_trend    0.123054
       part_of_day_night    0.093111
     load_volatility_24h    0.057877
load_same_hour_yesterday    0.051437


In [11]:
import plotly.graph_objects as go

def plot_daily_patterns(y_train):
    """Plot average load patterns by day of week"""
    # Create pivot table for daily patterns
    pivot_df = pd.pivot_table(
        data=pd.DataFrame({
            'load': y_train,
            'hour': y_train.index.hour,
            'weekday': y_train.index.strftime('%A')
        }),
        values='load',
        index='hour',
        columns='weekday',
        aggfunc='mean'
    )
    
    # Create figure
    fig = go.Figure()
    
    # Add traces for each day
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    for day in days_order:
        fig.add_trace(
            go.Scatter(
                x=pivot_df.index,
                y=pivot_df[day],
                name=day,
                mode='lines+markers'
            )
        )
    
    # Update layout
    fig.update_layout(
        template='plotly_dark',
        title='Average Load Patterns by Day of Week',
        xaxis_title='Hour of Day',
        yaxis_title='Average Load (MW)',
        height=500,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.05
        )
    )
    
    return fig

def plot_weekly_distribution(y_train):
    """Plot load distribution by day of week"""
    # Create figure
    fig = go.Figure()
    
    # Add box plots for each day
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    for day in days_order:
        day_data = y_train[y_train.index.strftime('%A') == day]
        fig.add_trace(
            go.Box(
                y=day_data,
                name=day
            )
        )
    
    # Update layout
    fig.update_layout(
        template='plotly_dark',
        title='Load Distribution by Day of Week',
        xaxis_title='Day of Week',
        yaxis_title='Load (MW)',
        height=500,
        showlegend=False
    )
    
    return fig

def plot_weekly_comparison(y_train):
    """Plot actual vs last week load comparison"""
    # Prepare data
    sample_period = y_train['2023-06-01':'2023-06-14']
    sample_lastweek = sample_period.shift(7*96)
    
    # Create figure
    fig = go.Figure()
    
    # Add traces
    fig.add_trace(
        go.Scatter(
            x=sample_period.index,
            y=sample_period,
            name='Actual Load',
            line=dict(color='#636EFA')
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=sample_period.index,
            y=sample_lastweek,
            name='Last Week Load',
            line=dict(color='#EF553B', dash='dash')
        )
    )
    
    # Update layout
    fig.update_layout(
        template='plotly_dark',
        title='Actual Load vs Last Week Load (Two Weeks Sample)',
        xaxis_title='Date',
        yaxis_title='Load (MW)',
        height=500
    )
    
    return fig

# Create and display all plots
def display_all_plots():
    """Display all three plots and print correlation"""
    # Create plots
    daily_fig = plot_daily_patterns(y_train)
    weekly_fig = plot_weekly_distribution(y_train)
    comparison_fig = plot_weekly_comparison(y_train)
    
    # Display plots
    daily_fig.show()
    weekly_fig.show()
    comparison_fig.show()
    
    # Print correlation
    correlation = y_train.corr(y_train.shift(7*96))
    print(f"\nCorrelation between current load and last week's load: {correlation:.3f}")

# Display all plots
display_all_plots()


Correlation between current load and last week's load: 0.919


In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_weekly_trend():
    """Visualize the weekly load trend"""
    # Calculate weekly trend
    weekly_mean = y_train.rolling(window=96*7).mean()
    
    # Sample period for visualization (2 months)
    sample_period = y_train['2023-06-01':'2023-07-31']
    sample_trend = weekly_mean['2023-06-01':'2023-07-31']
    
    fig = go.Figure()
    
    # Add actual load
    fig.add_trace(
        go.Scatter(
            x=sample_period.index,
            y=sample_period,
            name='Actual Load',
            line=dict(color='#636EFA', width=1),
            opacity=0.5
        )
    )
    
    # Add weekly trend
    fig.add_trace(
        go.Scatter(
            x=sample_trend.index,
            y=sample_trend,
            name='Weekly Trend',
            line=dict(color='#EF553B', width=3)
        )
    )
    
    fig.update_layout(
        template='plotly_dark',
        title='Weekly Load Trend (2-Month Sample)',
        xaxis_title='Date',
        yaxis_title='Load (MW)',
        height=500
    )
    
    return fig

def plot_night_pattern():
    """Visualize night period load patterns"""
    # Create hour-based average loads
    hourly_avg = pd.DataFrame({
        'load': y_train,
        'hour': y_train.index.hour,
        'is_night': y_train.index.hour.isin(range(22, 24)) | y_train.index.hour.isin(range(0, 6))
    }).groupby(['hour', 'is_night'])['load'].mean().reset_index()
    
    fig = go.Figure()
    
    # Add day hours
    day_data = hourly_avg[~hourly_avg['is_night']]
    fig.add_trace(
        go.Scatter(
            x=day_data['hour'],
            y=day_data['load'],
            name='Day Hours',
            mode='lines+markers',
            line=dict(color='#636EFA')
        )
    )
    
    # Add night hours
    night_data = hourly_avg[hourly_avg['is_night']]
    fig.add_trace(
        go.Scatter(
            x=night_data['hour'],
            y=night_data['load'],
            name='Night Hours (22-06)',
            mode='lines+markers',
            line=dict(color='#EF553B')
        )
    )
    
    fig.update_layout(
        template='plotly_dark',
        title='Average Load by Hour (Night vs Day)',
        xaxis_title='Hour of Day',
        yaxis_title='Average Load (MW)',
        height=500
    )
    
    return fig

def plot_load_volatility():
    """Visualize 24-hour load volatility"""
    # Calculate 24-hour volatility
    volatility = y_train.rolling(window=96).std()
    
    # Sample period for visualization
    sample_period = y_train['2023-06-01':'2023-06-14']
    sample_volatility = volatility['2023-06-01':'2023-06-14']
    
    fig = go.Figure()
    
    # Add actual load
    fig.add_trace(
        go.Scatter(
            x=sample_period.index,
            y=sample_period,
            name='Actual Load',
            line=dict(color='#636EFA'),
            yaxis='y'
        )
    )
    
    # Add volatility
    fig.add_trace(
        go.Scatter(
            x=sample_volatility.index,
            y=sample_volatility,
            name='24h Volatility',
            line=dict(color='#EF553B'),
            yaxis='y2'
        )
    )
    
    fig.update_layout(
        template='plotly_dark',
        title='Load and 24-Hour Volatility (2-Week Sample)',
        xaxis_title='Date',
        yaxis_title='Load (MW)',
        yaxis2=dict(
            title='Volatility',
            overlaying='y',
            side='right'
        ),
        height=500
    )
    
    return fig

def plot_yesterday_comparison():
    """Visualize yesterday vs today load patterns"""
    # Sample period for visualization
    sample_period = y_train['2023-06-01':'2023-06-07']
    yesterday_load = sample_period.shift(96)  # 24h * 4 (15-min intervals)
    
    fig = go.Figure()
    
    # Add actual load
    fig.add_trace(
        go.Scatter(
            x=sample_period.index,
            y=sample_period,
            name='Actual Load',
            line=dict(color='#636EFA')
        )
    )
    
    # Add yesterday's load
    fig.add_trace(
        go.Scatter(
            x=sample_period.index,
            y=yesterday_load,
            name="Yesterday's Load",
            line=dict(color='#EF553B', dash='dash')
        )
    )
    
    fig.update_layout(
        template='plotly_dark',
        title='Today vs Yesterday Load Comparison (1-Week Sample)',
        xaxis_title='Date',
        yaxis_title='Load (MW)',
        height=500
    )
    
    return fig

# Display all feature importance visualizations
def display_feature_importance_plots():
    """Display all feature importance visualization plots"""
    weekly_trend_fig = plot_weekly_trend()
    night_pattern_fig = plot_night_pattern()
    volatility_fig = plot_load_volatility()
    yesterday_fig = plot_yesterday_comparison()
    
    weekly_trend_fig.show()
    night_pattern_fig.show()
    volatility_fig.show()
    yesterday_fig.show()

# Create visualizations
display_feature_importance_plots()

# Feature Engineering for Load Forecasting
## German Energy Load Prediction Project

This notebook demonstrates the feature engineering process for predicting energy load 24 hours ahead using German energy data.

### 1. Initial Setup and Imports

In [1]:
# Standard library imports
import sys
import warnings
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import holidays
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning)

# Project path setup
current_dir = Path().absolute()
project_root = current_dir.parent if 'notebooks' in str(current_dir) else current_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
print(f"Project root added to path: {project_root}")

Project root added to path: d:\Projects\PORTF_2_German_Energy_forcast\german-energy-forcast


### 2. Load and Prepare Data

In [2]:
# Load actual_load data and handle timezones properly
actual_load_df = pd.read_csv('../data/raw/load_forecast_data_2years.csv', 
                           parse_dates=['Unnamed: 0'],
                           index_col='Unnamed: 0')

# Load renewables data and handle timezones properly
renewables_df = pd.read_csv('../data/raw/renewables_20230101_20240301.csv', 
                           parse_dates=['Unnamed: 0'],
                           index_col='Unnamed: 0')

# First convert to UTC, then remove timezone
renewables_df.index = pd.to_datetime(renewables_df.index, utc=True).tz_localize(None)

# Print column names to debug
print("Renewables columns:", renewables_df.columns.tolist())

ImportError: cannot import name 'LoadDataLoader' from 'src.data.data_loader' (d:\Projects\PORTF_2_German_Energy_forcast\german-energy-forcast\src\data\data_loader.py)

### 3. Initialize Feature Extractor

In [ ]:
from src.features.feature_engineering import EnhancedFeatureExtractor24h, train_specialized_models, predict_with_specialized_models

### 4. Extract Features

In [ ]:
# Initialize feature extractor
feature_extractor = EnhancedFeatureExtractor24h()

# Prepare input DataFrame
input_df = pd.DataFrame({'load': processed_data})
input_df = pd.concat([input_df, renewables_df], axis=1)

# Extract features
features_df = feature_extractor.extract_features(input_df)

print("\nFeature DataFrame Shape:", features_df.shape)
print("\nFeature Columns:", features_df.columns.tolist())

### 5. Examine Features 

In [ ]:
# Display first few rows
print("First few rows of features:")
display(features_df.head())

# Basic statistics
print("\nFeature Statistics:")
display(features_df.describe())

# 3. Model Selection and Initial Training

This initial modeling step will help in:
1. Establish a baseline performance
2. Compare different algorithms
3. Identify potential issues with the predictions